Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
import time
from pyspark.sql import functions as fun

In [0]:
filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = (spark.read
  .format("csv")
  .option("header","true")
  .option("inferSchema","true")
  .load(filePath))

current_epoch = int(time.time())            
namesDf = namesDf.withColumn("execution_time", fun.lit(current_epoch))


namesDf = namesDf.withColumn("height_feet", fun.round(namesDf['height'] * 0.0328, 2))


namesDf = namesDf.withColumn("first_name", fun.split(namesDf['name'], " ")[0])
the_most_popular_name = (namesDf
                         .groupBy('first_name')
                         .count()
                         .orderBy(fun.desc('count'))
                         .limit(1))
display(the_most_popular_name)


namesDf = namesDf.withColumn("date_of_birth",
    fun.coalesce(
        fun.to_date("date_of_birth", "yyyy-MM-dd"),
        fun.to_date("date_of_birth", "dd.MM.yyyy") 
    ))

namesDf = namesDf.withColumn("date_of_death",
    fun.coalesce(
        fun.to_date("date_of_death", "yyyy-MM-dd"),
        fun.to_date("date_of_death", "dd.MM.yyyy")
    ))

namesDf = namesDf.withColumn("age", fun.when(fun.col("date_of_death").isNotNull(), 
          fun.floor(fun.datediff("date_of_death", "date_of_birth") / 365.25))
          .otherwise(fun.floor(fun.datediff(fun.current_date(), "date_of_birth") / 365.25))
)


namesDf = namesDf.drop('bio', 'death_details')


for col in namesDf.columns:
  namesDf = namesDf.withColumnRenamed(col, col.capitalize().replace('_', ' '))


namesDf = namesDf.sort(fun.col('Name').asc())

display(namesDf.limit(20))
namesDf.explain(True)

first_name,count
John,873


Imdb name id,Name,Birth name,Height,Birth details,Date of birth,Place of birth,Date of death,Place of death,Reason of death,Spouses string,Spouses,Divorces,Spouses with children,Children,Execution time,Height feet,First name,Age
nm1001478,'Big' LeRoy Mobley,LeRoy King Mobley III,193,"April 1, 1973 in Atlantic City, New Jersey, USA",1973-04-01,"Atlantic City, New Jersey, USA",null,null,null,null,0,0,0,0,1742317962,6.33,'Big',51
nm0521811,'Ducky' Louie,Lawrence Louie,null,"July 22, 1931 in Berkeley, California, USA",1931-07-22,"Berkeley, California, USA",null,null,null,null,0,0,0,0,1742317962,null,'Ducky',93
nm0722372,'Little Billy' Rhodes,William H. Rhodes,null,"February 1, 1895 in Illinois, USA",1895-02-01,"Illinois, USA",1967-07-24,"Hollywood, California, USA",stroke,null,0,0,0,0,1742317962,null,'Little,72
nm0946148,'Weird Al' Yankovic,Alfred Matthew Yankovic,183,"October 23, 1959 in Downey, California, USA",1959-10-23,"Downey, California, USA",null,null,null,Suzanne Krajewski (10 February 2001 - present) (1 child),1,0,1,1,1742317962,6.0,'Weird,65
nm1265067,50 Cent,Curtis James Jackson III,183,"July 6, 1975 in Queens, New York City, New York, USA",1975-07-06,"Queens, New York City, New York, USA",null,null,null,null,0,0,0,0,1742317962,6.0,50,49
nm0553436,A Martinez,Adolph Larrue Martinez III,175,"September 27, 1948 in Glendale, California, USA",1948-09-27,"Glendale, California, USA",null,null,null,Leslie Bryans (17 July 1982 - present) (3 children)Mare Winningham (1981 - 29 January 1982) (divorced),2,1,1,3,1742317962,5.74,A,76
nm1100197,A. Baldwin Sloane,A. Baldwin Sloane,null,"August 28, 1872 in Baltimore, Maryland, USA",1872-08-28,"Baltimore, Maryland, USA",1925-02-21,"Red Bank, New Jersey, USA",null,null,0,0,0,0,1742317962,null,A.,52
nm0080406,A. Bhimsingh,A. Bhimsingh,null,"July 15, 1924 in Tirupati, Andhra Pradesh, India",1924-07-15,"Tirupati, Andhra Pradesh, India",1978-01-16,"Madras, Tamil Nadu, India",null,Sukumari (? - 16 January 1978) (his death) (1 child),1,0,1,1,1742317962,null,A.,53
nm0770661,A. Hans Scheirl,Angela Hans Schierl,null,"1956 in Salzburg, Austria",null,"Salzburg, Austria",null,null,null,null,0,0,0,0,1742317962,null,A.,null
nm0072200,A. Jonathan Benny,A. Jonathan Benny,null,"November 4, 1970",1970-11-04,null,null,null,null,null,0,0,0,0,1742317962,null,A.,54


== Parsed Logical Plan ==
'Sort ['Name ASC NULLS FIRST], true
+- Project [Imdb name id#11449, Name#11469, Birth name#11489, Height#11509, Birth details#11529, Date of birth#11549, Place of birth#11569, Date of death#11589, Place of death#11609, Reason of death#11629, Spouses string#11649, Spouses#11669, Divorces#11689, Spouses with children#11709, Children#11729, Execution time#11749, Height feet#11769, First name#11789, age#11408L AS Age#11809L]
   +- Project [Imdb name id#11449, Name#11469, Birth name#11489, Height#11509, Birth details#11529, Date of birth#11549, Place of birth#11569, Date of death#11589, Place of death#11609, Reason of death#11629, Spouses string#11649, Spouses#11669, Divorces#11689, Spouses with children#11709, Children#11729, Execution time#11749, Height feet#11769, first_name#11309 AS First name#11789, age#11408L]
      +- Project [Imdb name id#11449, Name#11469, Birth name#11489, Height#11509, Birth details#11529, Date of birth#11549, Place of birth#11569, Date 

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
moviesDf = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath))

current_epoch = int(time.time())
moviesDf = moviesDf.withColumn("execution_time", fun.lit(current_epoch))


moviesDf = moviesDf.withColumn('age', fun.when(
                                fun.col('year').isNotNull(), 
                                fun.year(fun.current_date()) - fun.col('year'))
                               .otherwise(None))


moviesDf = moviesDf.withColumn('budget_number', fun.regexp_replace
                               (fun.col('budget'), r'[^\d.]',''))

moviesDf = moviesDf.dropna()                               

display(moviesDf.limit(20))
moviesDf.explain(True)

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,execution_time,age,budget_number
tt0017136,Metropolis,Metropolis,1927,01.10.1928,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rogge, Fritz Rasp, Theodor Loos, Erwin Biswanger, Heinrich George, Brigitte Helm","In a futuristic city sharply divided between the working class and the city planners, the son of the city's mastermind falls in love with a working class prophet who predicts the coming of a savior to mediate their differences.",08.mar,156076,DEM 6000000,$ 1236166,$ 1349711,98.0,495.0,208.0,1742317980,98.0,6000000
tt0021749,Luci della città,City Lights,1931,02.04.1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, Al Ernest Garcia, Hank Mann, Charles Chaplin","With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.",08.maj,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0,1742317980,94.0,1500000
tt0027977,Tempi moderni,Modern Times,1936,12.03.1937,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergman, Tiny Sandford, Chester Conklin, Hank Mann, Stanley Blystone, Al Ernest Garcia, Richard Alexander, Cecil Reynolds, Mira McKinney, Murdock MacQuarrie, Wilfred Lucas, Edward LeSaint, Fred Malatesta",The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,08.maj,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0,1742317980,89.0,1500000
tt0029453,Il bandito della Casbah,Pépé le Moko,1937,22.10.1937,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fernand Charpin, Lucas Gridoux, Gilbert Gil, Marcel Dalio, Charles Granval, Gaston Modot, René Bergeron, Paul Escoffier, Roger Legris, Jean Témerson, Robert Ozanne, Philippe Richard","A wanted gangster is both king and prisoner of the Casbah. He is protected from arrest by his friends, but is torn by his desire for freedom outside. A visiting Parisian beauty may just tempt his fate.",07.lip,6180,$ 60000,$ 155895,$ 155895,98.0,46.0,55.0,1742317980,88.0,60000
tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,30.11.1938,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti, Eddie Collins, Pinto Colvig, Marion Darlington, Billy Gilbert, Otis Harlan, Lucille La Verne, James MacDonald, Scotty Mattraw, Moroni Olsen, Purv Pullen, Harry Stockwell, Bill Thompson","Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",07.cze,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,1742317980,88.0,1499000
tt0031381,Via col vento,Gone with the Wind,1939,12.03.1949,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh, Evelyn Keyes, Ann Rutherford, George Reeves, Fred Crane, Hattie McDaniel, Oscar Polk, Butterfly McQueen, Victor Jory, Everett Brown, Howard Hickman, Alicia Rhett, Leslie Howard",A manipulative woman and a roguish man conduct a turbulent romance during the American Civil War and Reconstruction periods.,08.sty,283975,$ 3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,1742317980,86.0,3977000
tt0031679,Mr. Smith va a Washington,Mr. Smith

== Parsed Logical Plan ==
Filter atleastnnonnulls(25, imdb_title_id#11903, title#11904, original_title#11905, year#11906, date_published#11907, genre#11908, duration#11909, country#11910, language#11911, director#11912, writer#11913, production_company#11914, actors#11915, description#11916, avg_vote#11917, votes#11918, budget#11919, usa_gross_income#11920, worlwide_gross_income#11921, metascore#11922, reviews_from_users#11923, reviews_from_critics#11924, execution_time#11947, ... 2 more fields)
+- Project [imdb_title_id#11903, title#11904, original_title#11905, year#11906, date_published#11907, genre#11908, duration#11909, country#11910, language#11911, director#11912, writer#11913, production_company#11914, actors#11915, description#11916, avg_vote#11917, votes#11918, budget#11919, usa_gross_income#11920, worlwide_gross_income#11921, metascore#11922, reviews_from_users#11923, reviews_from_critics#11924, execution_time#11947, age#11972, regexp_replace(budget#11919, [^\d.], , 1) AS bud

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
from pyspark.sql.types import LongType

In [0]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
ratingsDf = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath))

current_epoch = int(time.time())
ratingsDf = ratingsDf.withColumn("execution_time", fun.lit(current_epoch))

#avg automatycznie usunie nulla
males = ratingsDf.select(fun.avg('males_allages_avg_vote')).collect()[0][0]
females = ratingsDf.select(fun.avg('females_allages_avg_vote')).collect()[0][0]
display("males avg: ", males, " females avg: ", females)

ratingsDf = ratingsDf.withColumn(
    'execution_time', fun.when(ratingsDf['execution_time'].isNotNull(), 
    ratingsDf['execution_time'].cast(LongType())).otherwise(None)
)

display(ratingsDf.limit(20))
ratingsDf.explain(True)

'males avg: '5.817172176019753' females avg: '6.03940821227877

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,execution_time
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0,1742317990
tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,null,null,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0,1742317990
tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,null,null,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,null,null,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,null,null,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0,1742317990
tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,null,null,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,null,null,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,null,null,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0,1742317990
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0,1742317990
tt0002199,5.7,484,5.8,6.0,33,15,48,80,123,77,36,20,18,34,null,null,5.6,35.0,5.6,177.0,5.8,168.0,5.7,354.0,null,null,5.6,32.0,5.7,159.0,5.8,151.0,5.8,31.0,null,null,6.3,3.0,5.1,13.0,6.5,15.0,5.5,67.0,5.9,173.0,5.6,181.0,1742317990
tt0002423,6.8,753,6.8,7.0,80,65,105,209,142,80,27,13,4,28,3.0,1.0,6.4,59.0,6.7,287.0,7.0,276.0,6.6,530.0,3.0,1.0,6.4,41.0,6.6,240.0,6.8,238.0,7.4,93.0,null,null,6.5,16.0,7.2,40.0,8.2,34.0,6.2,88.0,6.6,139.0,6.8,455.0,1742317990
tt0002445,6.2,273,6.2,6.0,15,8,30,74,66,40,22,6,2,10,6.0,1.0,5.9,34.0,6.2,99.0,6.3,69.0,6.2,191.0,6.0,1.0,5.8,29.0,6.2,85.0,6.3,64.0,6.3,21.0,null,null,6.5,5.0,6.3,11.0,6.2,5.0,5.7,41.0,5.8,40.0,6.2,154.0,1742317990
tt0002452,6.7,198,7.1,7.0,53,7,23,35,30,24,12,5,3,6,null,null,7.4,28.0,6.7,104.0,6.5,39.0,6.7,153.0,null,null,7.3,24.0,6.6,92.0,6.5,33.0,7.0,19.0,null,null,8.3,3.0,6.8,10.0,6.8,6.0,5.4,25.0,6.7,25.0,6.8,135.0,1742317990
tt0002461,5.5,225,5.4,6.0,9,6,22,34,53,40,23,8,4,26,null,null,5.9,11.0,5.3,95.0,5.8,72.0,5.5,168.0,null,null,5.9,11.0,5.2,88.0,5.8,63.0,7.7,15.0,null,null,null,null,7.2,6.0,9.7,9.0,5.4,37.0,5.8,109.0,5.4,68.0,1742317990


== Parsed Logical Plan ==
'Project [imdb_title_id#12138, weighted_average_vote#12139, total_votes#12140, mean_vote#12141, median_vote#12142, votes_10#12143, votes_9#12144, votes_8#12145, votes_7#12146, votes_6#12147, votes_5#12148, votes_4#12149, votes_3#12150, votes_2#12151, votes_1#12152, allgenders_0age_avg_vote#12153, allgenders_0age_votes#12154, allgenders_18age_avg_vote#12155, allgenders_18age_votes#12156, allgenders_30age_avg_vote#12157, allgenders_30age_votes#12158, allgenders_45age_avg_vote#12159, allgenders_45age_votes#12160, males_allages_avg_vote#12161, ... 26 more fields]
+- Project [imdb_title_id#12138, weighted_average_vote#12139, total_votes#12140, mean_vote#12141, median_vote#12142, votes_10#12143, votes_9#12144, votes_8#12145, votes_7#12146, votes_6#12147, votes_5#12148, votes_4#12149, votes_3#12150, votes_2#12151, votes_1#12152, allgenders_0age_avg_vote#12153, allgenders_0age_votes#12154, allgenders_18age_avg_vote#12155, allgenders_18age_votes#12156, allgenders_30age